### Создаем соединения с БД

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Задаю пути для корректного поиска модулей внутри приложения
import os
import sys

src_path = os.getcwd()+'/src/'
data_path = os.getcwd()+'/data/'

sys.path.insert(0, src_path)
sys.path.insert(0, data_path)

In [2]:
from settings import *

ModuleNotFoundError: No module named 'src'

In [93]:
from ch_client import ClickHouse

from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [94]:
# mysql_client = Mysql(MYSQL_HOST_NAME, MYSQL_USER, MYSQL_PASS, MYSQL_DB_NAME)
ch_client = ClickHouse(CH_HOST_NAME, CH_CLIENT_PORT, CH_USER, CH_PASS, CH_DB_NAME)

In [95]:
ch_url = 'clickhouse://' + CH_USER + ':' + CH_PASS + '@' + CH_HOST_NAME + ':' + str(CH_HTTP_PORT) + '/' + CH_DB_NAME
ch_engine = create_engine('clickhouse://' + CH_USER + ':' + CH_PASS + '@' + CH_HOST_NAME + ':' + str(CH_HTTP_PORT) + '/' + CH_DB_NAME , echo=False)

# Обрабатываю данные для анализа

### ЗАДАЧА 1: Катя, 17 июля 2021: Конверсия переписки
Как участие переписки сотрудников влияет на создание заказов пользователем

In [96]:
date_start_project = pd.Timestamp('today')

# Имеет смысл оцениветь треды, созданные в течение последних 367 дней (больше года)
date_today = pd.to_datetime('today').date()
date_start = (date_today - pd.offsets.Day(367)).strftime("%Y-%m-%d")

# date_start = '2020-01-01'

date_start

'2020-08-24'

In [97]:
# Сообщения
sql = f''' SELECT DISTINCT mess.message_id,
                          mess.client_id as client_id,
                          mess.order_id,
                          user.group_name as user_group_name,
                          min(mess.created_at) as created_at,
                          count(DISTINCT mess.manager_id) as user_count,
                          count(mess.id) as user_message_count,
                          thread_birthday
            FROM mega_message as mess
            LEFT JOIN mega_user as user
                ON mess.manager_id = user.id
            WHERE 
                mess.manager_id is not null
                AND mess.order_id is null
                AND mess.thread_birthday >= toDate('{date_start}')
--              AND mess.client_id IN(136062, 167)
--              AND mess.manager_id = 7004
            GROUP BY mess.message_id,
                  mess.client_id,
                  user.group_name,
                  mess.order_id,
                  thread_birthday
            ORDER BY message_id
            FORMAT TabSeparatedWithNamesAndTypes'''
message = pd.read_sql_query(sql, ch_engine,)
message

,message_id,client_id,order_id,user_group_name,created_at,user_count,user_message_count,thread_birthday
0,1791277,364819,None,Техническая поддержка,2020-08-24 17:55:29,1,1,2020-08-24 00:03:53
1,1791281,402350,None,Техническая поддержка,2020-08-27 09:19:31,1,1,2020-08-24 00:18:48
2,1791282,172736,None,Техническая поддержка,2020-08-25 16:40:35,1,1,2020-08-24 00:21:13
3,1791284,304859,None,Менеджер по работе с клиентами,2020-08-24 09:44:32,1,1,2020-08-24 00:21:53
4,1791286,402351,None,Техническая поддержка,2020-08-25 16:40:07,1,1,2020-08-24 00:23:30
...,...,...,...,...,...,...,...,...
38509,2220970,472069,None,Менеджер по работе с клиентами,2021-07-15 09:20:43,1,1,2021-07-15 06:27:03
38510,2220977,472072,None,Менеджер по работе с клиентами,2021-07-15 09:24:31,1,1,2021-07-15 06:46:13
38511,2220980,472075,None,Менеджер по работе с клиентами,2021-07-15 09:18:09,1,1,2021-07-15 07:17:17
38512,2220996,472083,None,Менеджер по работе с клиентами,2021-07-15 09:13:27,1,1,2021-07-15 08:38:15


In [98]:
print(message.user_count.sum())
print(message.user_count.count())
print(message.user_message_count.sum())

40532
38514
48225


In [99]:
# Заказы
sql = f''' SELECT orders.id, 
                orders.client_id,
                orders.created_at 
            FROM mega_orders as orders
            WHERE 
                payed_summa >= summa
                AND orders.created_at >= toDate('{date_start}')
            ORDER BY client_id, created_at
            FORMAT TabSeparatedWithNamesAndTypes'''
orders = pd.read_sql(sql, ch_engine)
orders

,id,client_id,created_at
0,526334,111,2020-11-13 17:17:20
1,562474,111,2021-03-31 18:05:59
2,553502,126,2021-03-01 17:56:21
3,570691,126,2021-04-27 02:21:43
4,576466,126,2021-05-15 02:00:48
...,...,...,...
54665,596321,472082,2021-07-15 08:36:48
54666,596322,472084,2021-07-15 08:39:10
54667,596325,472086,2021-07-15 08:56:20
54668,596328,472089,2021-07-15 09:04:46


In [100]:
# В течение скольки дней после начала переписки клиент сделал заказ

message['days_to_order'] = 0
series_days = pd.Series([0])
for index, row in message.iterrows():
    message_created_at = row['created_at']
    message_client_id = row['client_id']
      
    next_order = orders \
                .query('client_id == @message_client_id & created_at >= @message_created_at')  \
                .sort_values('created_at', ascending=True) \
                .head(1)
    if len(next_order.index) > 0 :
        order_created_at = next_order.created_at.iloc[0]
        days_to_order = (order_created_at - message_created_at).days + 1
        if days_to_order > 366:
#           Отслеживаю только те заказы, которые появились в течение года
#           367 - условное число => заказ после переписки не зафиксирован
            days_to_order = 367
    else: 
        days_to_order = 367
    series_days[index] = days_to_order
print('ok')
message.days_to_order = series_days

ok


In [101]:
# После время-затратной операции делаю копию message
message_copy = message.copy()

In [102]:
# message.info()

In [103]:
message['order_7days'] = message['days_to_order'].apply(lambda x: 1 if x <= 7 else 0)
message['order_14days'] = message['days_to_order'].apply(lambda x: 1 if x <= 14 else 0)
message['order_21days'] = message['days_to_order'].apply(lambda x: 1 if x <= 21 else 0)
message['order_28days'] = message['days_to_order'].apply(lambda x: 1 if x <= 28 else 0)
message['order_365days'] = message['days_to_order'].apply(lambda x: 1 if x <= 366 else 0)
message['no_order'] = message['days_to_order'].apply(lambda x: 1 if x == 367 else 0)
message

,message_id,client_id,order_id,user_group_name,created_at,user_count,user_message_count,thread_birthday,days_to_order,order_7days,order_14days,order_21days,order_28days,order_365days,no_order
0,1791277,364819,None,Техническая поддержка,2020-08-24 17:55:29,1,1,2020-08-24 00:03:53,367,0,0,0,0,0,1
1,1791281,402350,None,Техническая поддержка,2020-08-27 09:19:31,1,1,2020-08-24 00:18:48,367,0,0,0,0,0,1
2,1791282,172736,None,Техническая поддержка,2020-08-25 16:40:35,1,1,2020-08-24 00:21:13,367,0,0,0,0,0,1
3,1791284,304859,None,Менеджер по работе с клиентами,2020-08-24 09:44:32,1,1,2020-08-24 00:21:53,367,0,0,0,0,0,1
4,1791286,402351,None,Техническая поддержка,2020-08-25 16:40:07,1,1,2020-08-24 00:23:30,367,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38509,2220970,472069,None,Менеджер по работе с клиентами,2021-07-15 09:20:43,1,1,2021-07-15 06:27:03,367,0,0,0,0,0,1
38510,2220977,472072,None,Менеджер по работе с клиентами,2021-07-15 09:24:31,1,1,2021-07-15 06:46:13,367,0,0,0,0,0,1
38511,2220980,472075,None,Менеджер по работе с клиентами,2021-07-15 09:18:09,1,1,2021-07-15 07:17:17,367,0,0,0,0,0,1
38512,2220996,472083,None,Менеджер по работе с клиентами,2021-07-15 09:13:27,1,1,2021-07-15 08:38:15,367,0,0,0,0,0,1


In [104]:
message['order_7days'].value_counts()

0    36576
1     1938
Name: order_7days, dtype: int64

In [105]:
message \
    .query('order_id != order_id') \
    .groupby(['user_group_name']) \
    .aggregate({'order_7days': 'sum',
               'order_14days': 'sum',
               'order_21days': 'sum',
               'order_28days': 'sum',
               'order_365days': 'sum',
               'no_order': 'sum'}) \
    .sort_values(['user_group_name'],
                 ascending=[False]) 

,order_7days,order_14days,order_21days,order_28days,order_365days,no_order
user_group_name,,,,,,
Техническая поддержка,1021,1269,1409,1501,2182,15540
Специалист контактного центра,264,338,388,427,678,4480
Менеджер по работе с поставщиками,0,0,1,1,4,63
Менеджер по работе с клиентами,653,865,1020,1127,1773,13794


In [106]:
message['message_id'].size

38514

In [107]:
with ch_engine.connect() as connection:
    with connection.begin():
        r1 = connection.execute(f'''ALTER TABLE mega_message_thread_by_user_group 
                                    DELETE WHERE thread_birthday>=toDate('{date_start}')''')

In [108]:
message.to_sql('mega_message_thread_by_user_group', ch_engine, if_exists='append', index=False)

### Логи

In [109]:
from logs import set_log

In [110]:
date_end_project = pd.Timestamp('today')
status = 'OK'
rows = message['message_id'].size

log = f'''Обновлены данные по тредам созданным с {date_start}: {rows} шт.'''

In [111]:
set_log(ch_engine, PROJECT, TASK, date_start_project, date_end_project, status, log)

In [2]:
import papermill